In [ ]:
!pip install numpy==1.24.4 --force

In [ ]:
from IPython.display import clear_output

!pip install cohere
!pip install umap-learn

clear_output()

In [ ]:
import cohere
import numpy as np
import requests
import umap

from google.colab import userdata
from pickle import loads

co = cohere.Client(userdata.get('COHERE_API_KEY'))
response = requests.get('https://both.rocks/maquinito-15/upniverso_umap.pkl')

if response.status_code != 200:
  print("ERROR: no existe el fichero upniverso_umap.pkl")
  print(response)

reducer = loads(response.content)


In [ ]:
verso = "El patio de mi casa"

In [ ]:
response = co.embed(texts=[verso], model="embed-multilingual-v2.0")
embeddings_768 = response.embeddings[0]
embeddings_3 = reducer.transform([embeddings_768])[0]

In [ ]:
print(embeddings_768)
print(embeddings_3)